El presente notebook se ha diseñado para la carga y uso del modelo ya entrenado, de manera que pueda comprobarse su funcionamiento

In [ ]:
!pip3 install protobuf
!pip3 install evaluate

In [22]:
# Se realizan aquí las importaciones
import import_ipynb
import selenium_tests
import yaml
from screeninfo import get_monitors
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import evaluate

In [2]:
# Dado que el Software implementado está diseñado para pantallas de 4K, se obtienen con ayuda de esta función las pantallas disponibles y su resolución
def obtener_info_pantallas():
    monitores = get_monitors()

    propiedades_monitor = None
    ancho_monitor = 0
    alto_monitor = 0
    i = 0
    while i < len(monitores):
        if monitores[i].width > ancho_monitor and monitores[i].height > alto_monitor:
            propiedades_monitor = monitores[i]

        i = i + 1

    return propiedades_monitor

In [26]:
# Se obtienen las propiedades del monitor y Se crea un objeto de Tests, a cuyo constructor pasamos como parámetro las propiedades del monitor con mayor resolución
#propiedades_monitor = obtener_info_pantallas()
#tests = selenium_tests.Tests(propiedades_monitor)

# Se configura el driver
#tests.configurar_driver()

# Se inicia la sesión
#tests.iniciar_sesion(params={})

#nombre_modelo = '.\\PLANTL-GOB-ES\\roberta-large-bne-massive-custom-lora'
nombre_modelo = 'roberta-large-bne-massive_peft_fusionado'
#nombre_tokenizador = 'roberta-large-bne-massive_peft_fusionado'
nombre_tokenizador = './PLANTL-GOB-ES/roberta-large-bne-massive-custom-lora'

#nombre_modelo = 'albert-base-10-spanish-finetuned-mldoc_peft_fusionado'
#nombre_tokenizador = 'albert-base-10-spanish-finetuned-mldoc_peft_fusionado'
#evaluate.load("accuracy")
modelo_roberta = AutoModelForSequenceClassification.from_pretrained(nombre_modelo, local_files_only=True)
tokenizador_roberta = AutoTokenizer.from_pretrained(nombre_tokenizador, cache_dir='./huggingface_mirror', local_files_only=True, from_pt=True)
pipe = pipeline('text-classification', modelo_roberta, tokenizer=tokenizador_roberta)

try:
    while True:
        prueba_a_realizar = input('\nIndique qué desea hacer: ')
        resultado = pipe(prueba_a_realizar)
        puntuacion = resultado[0]['score']
        print('Puntuación:', puntuacion)
        prueba = resultado[0]['label']
        
        # Se comprueba si se trata de una o de varias pruebas
        listado_pruebas = []
        if prueba.startswith('[') is False:
            print('ENTRA EN 1')
            prueba_yaml = yaml.safe_load(prueba)
            print(prueba_yaml)
            listado_pruebas.append(prueba_yaml)
            
        else:
            prueba = prueba[1:-1]
            listado_pruebas = prueba.split('\nURL:')

            # Se eliminan posibles entradas sin información de la lista debido al split
            while '' in listado_pruebas:
                listado_pruebas.remove('')

            # Se genera la lista con los YAML y crea el diccionario correspondiente para cada una de las pruebas a realizar
            listado_pruebas = ['\nURL:' + prueba for prueba in listado_pruebas]
            listado_pruebas = [yaml.safe_load(prueba) for prueba in listado_pruebas]
            
        for prueba_a_realizar in listado_pruebas:
            # Se va en primer lugar a la URL correspondiente. De ser la misma en que se encontraba no se ejecutarán acciones adicionales
            #tests.ir_a_url(prueba_a_realizar['URL'])

            # Se ejecutan a continuación las acciones requeridas
            for accion in prueba_a_realizar['acciones']:
                print('ACCION:', accion)
                #funcion = getattr(tests, accion['funcion'])
                #funcion(accion['params'])
            
        
except KeyboardInterrupt:
    pass
    #tests.cerrar_sesion()

Device set to use cpu



Indique qué desea hacer:  campo S


0.011585203930735588
[
URL: https://impact-neo-ced.enaire.es/#/main/master
acciones:
  - funcion: hacer_clic
    params: 
      - xpath: //div[contains(@class,"tab-ws")]/div/span[contains(text(),"OPER") or contains(text(),"FCST") or contains(text(),"SIMU")]
      
  - funcion: hacer_clic
    params: 
      - xpath: //span[contains(@class, "hist-cell ifr-arr")]
      
  - funcion: comprobar_contenido_hover
    params: 
      - xpath: //div[@id="Flightlist"]//div[text()="D.T/L/P"]
        esperar: //*[contains(text(), "related")]
        comprobar:
          - xpath: //*[contains(text(), "Delta Time, Vertical Delta and horizontal Delta related to intrusion")]
        tipo_comprobacion: y
] <class 'str'>
ACCION: {'funcion': 'hacer_clic', 'params': [{'xpath': '//div[contains(@class,"tab-ws")]/div/span[contains(text(),"OPER") or contains(text(),"FCST") or contains(text(),"SIMU")]'}]}
ACCION: {'funcion': 'hacer_clic', 'params': [{'xpath': '//span[contains(@class, "hist-cell ifr-arr")]'}]}
AC